In [ ]:
%pip install snowflake-connector-python

In [ ]:
import os
# Leer variables de entorno
SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT')
SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
SNOWFLAKE_DATABASE = os.getenv('SNOWFLAKE_DATABASE')
SNOWFLAKE_SCHEMA_ANALYTICS = os.getenv('SNOWFLAKE_SCHEMA_ANALYTICS', 'ANALYTICS')
SNOWFLAKE_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH')

In [ ]:
# CONFIGURACIÓN INICIAL PARA VALIDACIONES
print("=== VALIDACIONES Y EXPLORACIÓN - OBT NYC TAXI ===")

# Conexión a Snowflake (reutilizar configuración del notebook anterior)
try:
    import snowflake.connector
    import time
    from datetime import datetime

    print("snowflake-connector-python disponible")

    # Configuración de conexión
    conn_config = {
        'user': SNOWFLAKE_USER,
        'password': SNOWFLAKE_PASSWORD,
        'account': SNOWFLAKE_ACCOUNT,
        'warehouse': SNOWFLAKE_WAREHOUSE,
        'database': SNOWFLAKE_DATABASE,
        'schema': SNOWFLAKE_SCHEMA_ANALYTICS
    }

    def execute_snowflake_sql(sql_query, fetch_results=True):
        """
        Ejecuta SQL directamente en Snowflake con resultados
        """
        try:
            conn = snowflake.connector.connect(**conn_config)
            cursor = conn.cursor()

            cursor.execute(sql_query)

            if fetch_results:
                results = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]
                return results, columns
            else:
                rows_affected = cursor.rowcount
                print(f" SQL ejecutado. Filas afectadas: {rows_affected:,}")
                return rows_affected, []

        except Exception as e:
            print(f"Error ejecutando SQL: {e}")
            return None, []
        finally:
            try:
                cursor.close()
                conn.close()
            except:
                pass

    def print_validation_results(title, results, columns):
        """Helper para mostrar resultados de validación"""
        print(f"\n{title}")
        print("=" * len(title))

        if not results:
            print("No se obtuvieron resultados")
            return

        # Imprimir resultados
        for row in results:
            if len(columns) == 2:  # Formato métrica: valor
                metric, value = row
                if isinstance(value, (int, float)):
                    if value == 0:
                        print(f"{metric}: {value:,}")
                    else:
                        print(f" {metric}: {value:,}")
                else:
                    print(f"{metric}: {value}")
            else:
                # Múltiples columnas
                print("   " + " | ".join([f"{col}: {val}" for col, val in zip(columns, row)]))

    print("Funciones de conexión y validación configuradas")

except ImportError:
    print("snowflake-connector-python no instalado")
    print("Instalar con: pip install snowflake-connector-python")

    def execute_snowflake_sql(sql_query, fetch_results=True):
        print("Connector no disponible. SQL para ejecutar manualmente:")
        print("-" * 60)
        print(sql_query)
        print("-" * 60)
        return None, []

    def print_validation_results(title, results, columns):
        print(f"{title}: Función no disponible sin connector")

print(" Configuración lista para validaciones en analytics.obt_trips")

=== VALIDACIONES Y EXPLORACIÓN - OBT NYC TAXI ===
snowflake-connector-python disponible
Funciones de conexión y validación configuradas
 Configuración lista para validaciones en analytics.obt_trips


## Validacion de nulos

In [5]:
# VALIDACIÓN 1: ANÁLISIS DE VALORES NULOS
print(" VALIDACIÓN 1: Análisis de Valores Nulos")
print("=" * 50)

# SQL para contar nulos en campos críticos
null_validation_sql = """
SELECT
    'trip_id' as campo,
    COUNT(*) - COUNT(trip_id) as nulos,
    ROUND((COUNT(*) - COUNT(trip_id)) * 100.0 / COUNT(*), 2) as pct_nulos
FROM analytics.obt_trips

UNION ALL

SELECT
    'pickup_datetime',
    COUNT(*) - COUNT(pickup_datetime),
    ROUND((COUNT(*) - COUNT(pickup_datetime)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'dropoff_datetime',
    COUNT(*) - COUNT(dropoff_datetime),
    ROUND((COUNT(*) - COUNT(dropoff_datetime)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'pu_zone',
    COUNT(*) - COUNT(pu_zone),
    ROUND((COUNT(*) - COUNT(pu_zone)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'do_zone',
    COUNT(*) - COUNT(do_zone),
    ROUND((COUNT(*) - COUNT(do_zone)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'service_type',
    COUNT(*) - COUNT(service_type),
    ROUND((COUNT(*) - COUNT(service_type)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'total_amount',
    COUNT(*) - COUNT(total_amount),
    ROUND((COUNT(*) - COUNT(total_amount)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'trip_distance',
    COUNT(*) - COUNT(trip_distance),
    ROUND((COUNT(*) - COUNT(trip_distance)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'avg_speed_mph',
    COUNT(*) - COUNT(avg_speed_mph),
    ROUND((COUNT(*) - COUNT(avg_speed_mph)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

UNION ALL

SELECT
    'tip_pct',
    COUNT(*) - COUNT(tip_pct),
    ROUND((COUNT(*) - COUNT(tip_pct)) * 100.0 / COUNT(*), 2)
FROM analytics.obt_trips

ORDER BY nulos DESC;
"""

try:
    results, columns = execute_snowflake_sql(null_validation_sql)

    print("ANÁLISIS DE NULOS POR CAMPO:")
    print("-" * 40)

    if results:
        for row in results:
            campo, nulos, pct_nulos = row
            if nulos == 0:
                print(f"{campo}: {nulos:,} nulos ({pct_nulos}%)")
            elif pct_nulos < 5:
                print(f"{campo}: {nulos:,} nulos ({pct_nulos}%) - Aceptable")
            else:
                print(f"{campo}: {nulos:,} nulos ({pct_nulos}%) - Crítico")

    print("\nINTERPRETACIÓN:")
    print("• 0% nulos: Campo completo")
    print("• <5% nulos: Aceptable para campos opcionales")
    print("• >5% nulos: Requiere investigación")

except Exception as e:
    print(f"Error en validación de nulos: {e}")

print("\n" + "="*50)

 VALIDACIÓN 1: Análisis de Valores Nulos
ANÁLISIS DE NULOS POR CAMPO:
----------------------------------------
pu_zone: 9,080,986 nulos (1.08%) - Aceptable
do_zone: 7,908,067 nulos (0.94%) - Aceptable
avg_speed_mph: 4,466,224 nulos (0.53%) - Aceptable
trip_id: 0 nulos (0.00%)
pickup_datetime: 0 nulos (0.00%)
dropoff_datetime: 0 nulos (0.00%)
service_type: 0 nulos (0.00%)
total_amount: 0 nulos (0.00%)
trip_distance: 0 nulos (0.00%)
tip_pct: 0 nulos (0.00%)

INTERPRETACIÓN:
• 0% nulos: Campo completo
• <5% nulos: Aceptable para campos opcionales
• >5% nulos: Requiere investigación



## Validacion de rangos y valores extremos


In [6]:
# VALIDACIÓN 2: ANÁLISIS DE RANGOS Y VALORES EXTREMOS
print("VALIDACIÓN 2: Análisis de Rangos y Valores Extremos")
print("=" * 60)

# SQL para validar rangos en campos numéricos
range_validation_sql = """
SELECT
    'trip_distance negativa' as validacion,
    COUNT(*) as registros_problematicos,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2) as porcentaje
FROM analytics.obt_trips
WHERE trip_distance < 0

UNION ALL

SELECT
    'trip_distance excesiva (>200 mi)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE trip_distance > 200

UNION ALL

SELECT
    'total_amount negativo',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE total_amount < 0

UNION ALL

SELECT
    'total_amount excesivo (>$1000)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE total_amount > 1000

UNION ALL

SELECT
    'passenger_count = 0',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE passenger_count = 0

UNION ALL

SELECT
    'passenger_count > 6',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE passenger_count > 6

UNION ALL

SELECT
    'velocidad > 150 mph',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE avg_speed_mph > 150

UNION ALL

SELECT
    'duración > 24 horas',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE trip_duration_min > 1440

UNION ALL

SELECT
    'duración < 1 minuto',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE trip_duration_min < 1

ORDER BY registros_problematicos DESC;
"""

try:
    results, columns = execute_snowflake_sql(range_validation_sql)

    print("VALIDACIONES DE RANGOS:")
    print("-" * 40)

    if results:
        total_problems = 0
        for row in results:
            validacion, problemas, porcentaje = row
            if problemas == 0:
                print(f"{validacion}: {problemas:,} casos ({porcentaje}%)")
            elif porcentaje < 1:
                print(f"{validacion}: {problemas:,} casos ({porcentaje}%) - Revisar")
            else:
                print(f"{validacion}: {problemas:,} casos ({porcentaje}%) - Crítico")
            total_problems += problemas

        print(f"\nRESUMEN: {total_problems:,} registros con problemas de rango")

except Exception as e:
    print(f"Error en validación de rangos: {e}")

print("\n" + "="*60)

VALIDACIÓN 2: Análisis de Rangos y Valores Extremos
VALIDACIONES DE RANGOS:
----------------------------------------
passenger_count = 0: 5,733,919 casos (0.68%) - Revisar
velocidad > 150 mph: 145,158 casos (0.02%) - Revisar
trip_distance excesiva (>200 mi): 12,527 casos (0.00%) - Revisar
passenger_count > 6: 5,342 casos (0.00%) - Revisar
total_amount excesivo (>$1000): 724 casos (0.00%) - Revisar
trip_distance negativa: 0 casos (0.00%)
duración > 24 horas: 0 casos (0.00%)
duración < 1 minuto: 0 casos (0.00%)
total_amount negativo: 0 casos (0.00%)

RESUMEN: 5,897,670 registros con problemas de rango



## Coherencia de fechas

In [7]:
# VALIDACIÓN 3: COHERENCIA TEMPORAL Y DE FECHAS
print(" VALIDACIÓN 3: Coherencia Temporal y de Fechas")
print("=" * 55)

# SQL para validar coherencia temporal
date_coherence_sql = """
SELECT
    'dropoff antes que pickup' as validacion,
    COUNT(*) as registros_problematicos,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2) as porcentaje
FROM analytics.obt_trips
WHERE dropoff_datetime <= pickup_datetime

UNION ALL

SELECT
    'pickup_date diferente a DATE(pickup_datetime)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE pickup_date != DATE(pickup_datetime)

UNION ALL

SELECT
    'pickup_hour diferente a HOUR(pickup_datetime)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE pickup_hour != HOUR(pickup_datetime)

UNION ALL

SELECT
    'day_of_week incoherente',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE day_of_week != DAYOFWEEK(pickup_datetime)

UNION ALL

SELECT
    'month incoherente',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE month != MONTH(pickup_datetime)

UNION ALL

SELECT
    'year incoherente',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE year != YEAR(pickup_datetime)

UNION ALL

SELECT
    'is_weekend incoherente',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE is_weekend != (DAYOFWEEK(pickup_datetime) IN (1, 7))

UNION ALL

SELECT
    'fechas futuras (>hoy)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE pickup_date > CURRENT_DATE()

UNION ALL

SELECT
    'fechas muy antiguas (<2015)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE pickup_date < '2015-01-01'

UNION ALL

SELECT
    'fechas muy futuras (>2025)',
    COUNT(*),
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM analytics.obt_trips), 2)
FROM analytics.obt_trips
WHERE pickup_date > '2025-12-31'

ORDER BY registros_problematicos DESC;
"""

try:
    results, columns = execute_snowflake_sql(date_coherence_sql)

    print(" VALIDACIONES DE COHERENCIA TEMPORAL:")
    print("-" * 45)

    if results:
        total_temporal_issues = 0
        for row in results:
            validacion, problemas, porcentaje = row
            if problemas == 0:
                print(f" {validacion}: {problemas:,} casos ({porcentaje}%)")
            elif porcentaje < 0.1:
                print(f"  {validacion}: {problemas:,} casos ({porcentaje}%) - Mínimo")
            else:
                print(f" {validacion}: {problemas:,} casos ({porcentaje}%) - Crítico")
            total_temporal_issues += problemas

        print(f"\n RESUMEN: {total_temporal_issues:,} problemas de coherencia temporal")

except Exception as e:
    print(f" Error en validación de coherencia: {e}")

# Análisis de rango temporal general
print("\n ANÁLISIS DE RANGO TEMPORAL:")
print("-" * 35)

temporal_range_sql = """
SELECT
    MIN(pickup_date) as fecha_minima,
    MAX(pickup_date) as fecha_maxima,
    DATEDIFF('day', MIN(pickup_date), MAX(pickup_date)) as dias_total,
    COUNT(DISTINCT pickup_date) as dias_con_datos,
    COUNT(DISTINCT CONCAT(year, '-', LPAD(month, 2, '0'))) as meses_con_datos
FROM analytics.obt_trips;
"""

try:
    results, columns = execute_snowflake_sql(temporal_range_sql)

    if results:
        row = results[0]
        fecha_min, fecha_max, dias_total, dias_con_datos, meses_con_datos = row

        print(f" Período detectado: {fecha_min} a {fecha_max}")
        print(f" Rango válido esperado: 2015-2025")
        print(f" Días totales: {dias_total:,}")
        print(f" Días con datos: {dias_con_datos:,}")
        print(f" Meses con datos: {meses_con_datos:,}")

        # Advertencia si hay fechas fuera de rango
        from datetime import date
        if fecha_min and (fecha_min < date(2015, 1, 1) or fecha_max > date(2025, 12, 31)):
            print(f" ¡ADVERTENCIA! Fechas fuera del rango válido 2015-2025")

        cobertura = (dias_con_datos / dias_total) * 100 if dias_total > 0 else 0
        print(f" Cobertura temporal: {cobertura:.1f}%")

except Exception as e:
    print(f" Error en análisis temporal: {e}")

print("\n" + "="*55)

 VALIDACIÓN 3: Coherencia Temporal y de Fechas
 VALIDACIONES DE COHERENCIA TEMPORAL:
---------------------------------------------
  fechas muy antiguas (<2015): 3,004 casos (0.00%) - Mínimo
  fechas muy futuras (>2025): 47 casos (0.00%) - Mínimo
  fechas futuras (>hoy): 47 casos (0.00%) - Mínimo
 pickup_hour diferente a HOUR(pickup_datetime): 0 casos (0.00%)
 dropoff antes que pickup: 0 casos (0.00%)
 month incoherente: 0 casos (0.00%)
 year incoherente: 0 casos (0.00%)
 is_weekend incoherente: 0 casos (0.00%)
 day_of_week incoherente: 0 casos (0.00%)
 pickup_date diferente a DATE(pickup_datetime): 0 casos (0.00%)

 RESUMEN: 3,098 problemas de coherencia temporal

 ANÁLISIS DE RANGO TEMPORAL:
-----------------------------------
 Período detectado: 2001-01-01 a 2098-09-11
 Rango válido esperado: 2015-2025
 Días totales: 35,682
 Días con datos: 3,955
 Meses con datos: 174
 ¡ADVERTENCIA! Fechas fuera del rango válido 2015-2025
 Cobertura temporal: 11.1%



## Conteos por mes y servicio

In [8]:
# VALIDACIÓN 4: DISTRIBUCIÓN POR MES Y SERVICIO
print("VALIDACIÓN 4: Distribución por Mes y Servicio")
print("=" * 55)

# Análisis por tipo de servicio
print(" DISTRIBUCIÓN POR TIPO DE SERVICIO:")
print("-" * 40)

service_distribution_sql = """
SELECT
    service_type,
    COUNT(*) as total_viajes,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) as porcentaje,
    ROUND(AVG(total_amount), 2) as tarifa_promedio,
    ROUND(AVG(trip_distance), 2) as distancia_promedio,
    ROUND(AVG(tip_pct), 1) as propina_promedio_pct
FROM analytics.obt_trips
GROUP BY service_type
ORDER BY total_viajes DESC;
"""

try:
    results, columns = execute_snowflake_sql(service_distribution_sql)

    if results:
        for row in results:
            service, viajes, pct, tarifa, distancia, propina = row
            print(f"{service.upper()}:")
            print(f"   Viajes: {viajes:,} ({pct}%)")
            print(f"   Tarifa promedio: ${tarifa}")
            print(f"   Distancia promedio: {distancia} mi")
            print(f"   Propina promedio: {propina}%")
            print()

except Exception as e:
    print(f"Error en distribución por servicio: {e}")

# Análisis por mes
print("DISTRIBUCIÓN POR MES:")
print("-" * 30)

monthly_distribution_sql = """
SELECT
    year,
    month,
    month_name,
    COUNT(*) as total_viajes,
    COUNT(CASE WHEN service_type = 'yellow' THEN 1 END) as yellow_trips,
    COUNT(CASE WHEN service_type = 'green' THEN 1 END) as green_trips,
    ROUND(AVG(total_amount), 2) as tarifa_promedio,
    ROUND(SUM(total_amount), 2) as revenue_total
FROM analytics.obt_trips
GROUP BY year, month, month_name
ORDER BY year, month;
"""

try:
    results, columns = execute_snowflake_sql(monthly_distribution_sql)

    if results:
        total_revenue = 0
        print("Año | Mes | Viajes Total | Yellow | Green | Tarifa Prom | Revenue")
        print("-" * 70)

        years_with_errors = []
        for row in results:
            year, month, month_name, total, yellow, green, tarifa, revenue = row
            total_revenue += revenue or 0

            # Detectar años fuera del rango válido
            if year < 2015 or year > 2025:
                years_with_errors.append(year)
                print(f"{year} | {month:2d} | {total:,} | {yellow:,} | {green:,} | ${tarifa} | ${revenue:,.0f} ← AÑO ERRÓNEO!")
            else:
                print(f"{year} | {month:2d} | {total:,} | {yellow:,} | {green:,} | ${tarifa} | ${revenue:,.0f}")

        print("-" * 70)
        print(f" REVENUE TOTAL: ${total_revenue:,.2f}")

        # Mostrar resumen de años erróneos
        if years_with_errors:
            unique_error_years = sorted(set(years_with_errors))
            print(f" AÑOS ERRÓNEOS DETECTADOS: {unique_error_years}")
            print(f" Total registros con años inválidos: {len(years_with_errors):,}")
            print(" Estos datos requieren limpieza - están fuera del rango 2015-2025")
        else:
            print(" Todos los años están en el rango válido 2015-2025")

except Exception as e:
    print(f" Error en distribución mensual: {e}")

# Análisis de consistencia temporal
print("\n CONSISTENCIA DE CONTEOS:")
print("-" * 35)

consistency_sql = """
SELECT
    'Total general' as metrica,
    COUNT(*) as valor
FROM analytics.obt_trips

UNION ALL

SELECT
    'Con trip_id único',
    COUNT(DISTINCT trip_id)
FROM analytics.obt_trips

UNION ALL

SELECT
    'Con natural_key',
    COUNT(DISTINCT natural_key)
FROM analytics.obt_trips

UNION ALL

SELECT
    'Yellow taxi',
    COUNT(*)
FROM analytics.obt_trips
WHERE service_type = 'yellow'

UNION ALL

SELECT
    'Green taxi',
    COUNT(*)
FROM analytics.obt_trips
WHERE service_type = 'green';
"""

try:
    results, columns = execute_snowflake_sql(consistency_sql)

    if results:
        for row in results:
            metrica, valor = row
            print(f" {metrica}: {valor:,}")

except Exception as e:
    print(f" Error en análisis de consistencia: {e}")

print("\n" + "="*55)

VALIDACIÓN 4: Distribución por Mes y Servicio
 DISTRIBUCIÓN POR TIPO DE SERVICIO:
----------------------------------------
YELLOW:
   Viajes: 776,718,444 (92.18%)
   Tarifa promedio: $18.73
   Distancia promedio: 5.66 mi
   Propina promedio: 17.2%

GREEN:
   Viajes: 65,924,389 (7.82%)
   Tarifa promedio: $15.79
   Distancia promedio: 7.14 mi
   Propina promedio: 9.8%

DISTRIBUCIÓN POR MES:
------------------------------
Año | Mes | Viajes Total | Yellow | Green | Tarifa Prom | Revenue
----------------------------------------------------------------------
2001 |  1 | 23 | 23 | 0 | $25.93 | $596 ← AÑO ERRÓNEO!
2001 |  2 | 1 | 1 | 0 | $3.8 | $4 ← AÑO ERRÓNEO!
2001 |  8 | 1 | 1 | 0 | $24.55 | $25 ← AÑO ERRÓNEO!
2002 |  2 | 11 | 11 | 0 | $17.9 | $197 ← AÑO ERRÓNEO!
2002 | 10 | 234 | 234 | 0 | $22.58 | $5,283 ← AÑO ERRÓNEO!
2002 | 12 | 43 | 43 | 0 | $31.06 | $1,336 ← AÑO ERRÓNEO!
2003 |  1 | 35 | 35 | 0 | $35.42 | $1,240 ← AÑO ERRÓNEO!
2003 |  3 | 1 | 1 | 0 | $3.31 | $3 ← AÑO ERRÓNEO!
2003 |